# Verification and Synthesis by UCLID5, the Huffman Encoding Hardware Implementation

Kai and Mingsheng

## Outline

1. Model Introduction
2. UCLID5 Basics
3. Verification
4. Synthesis

## Model Introduction

### Problem
Given an input text, count different characters and their frequenciies and come up with an encoding such that the text size is minimized.

### Example: "11110000 >> 111 ="

| character | frequency |
| --------- | --------- |
| '1'       | 5         |
| '0'       | 4         |
| '\s'      | 3         |
| '>'       | 2         |
| '='       | 1         |

- The easy way to encode **5** characters needs **3 bits** for each.
- The total text size would be **3\*(5+4+3+2+1) = 45**

\**This slide is taken from our CSE225 presentation*

### Solution

Iteratively combine the two least frequent characters into a group.

<img src="huff1.PNG" style="width: 600px;"/>

\**This slide is taken from our CSE225 presentation*

<img src="huff2.PNG" style="width: 600px;"/>

\**This slide is taken from our CSE225 presentation*

<img src="huff3.PNG" style="width: 650px;"/>

\**This slide is taken from our CSE225 presentation*

<img src="huff4.PNG" style="width: 600px;"/>

The resulting tree provides the optimal encoding:
- 2\*5 + 2\*4 + 2\*3 + 3\*2 + 3\*1 = **33** < 45

\**This slide is taken from our CSE225 presentation*

### Environment: Huffman Node Memory

Pre-populated with the characters and frequencies.
- The **<span style="color:blue">addr 0 in blue</span>** is reserved for denoting NULL
- The **<span style="color:red">addrs in red</span>** stores leaf nodes (the character-frequency pairs). Note that some tailing entries might be empty
- The **<span style="color:green">rest in green</span>** will store internal nodes. Note that some tailing entries might be empty
- The left/right addr encodes the tree
- Three read ports for same cycle access to parent and children

| addr          | char  | frequency | left child addr | right child addr |
| ------------- | ----- | --------- | --------------- | ---------------- |
| **<span style="color:blue">0</span>**             |       |           |                 |                  |
| **<span style="color:red">1</span>**              | '1'   | 5         | 0               | 0                |
| **<span style="color:red">2</span>**              | '0'   | 4         | 0               | 0                |
| **<span style="color:red">3</span>**              | '\\s' | 3         | 0               | 0                |
| **<span style="color:red">...</span>**           |       |           |                 |                  |
| **<span style="color:red">ALPHABET_SIZE</span>** |       |           |                 |                  |
| **<span style="color:green">ALPHABET_SIZE+1</span>** |       |           |                 |                  |
| **<span style="color:green">...</span>**           |       |           |                 |                  |
| **<span style="color:green">MAX_NODE</span>**      |       |           |                 |                  |

\**This slide is taken from our CSE225 presentation*

### System: Huffman Constructor

A state machine that builds the Huffman tree.
- Read characters and frequencies from the memory (read leaves)
- Write new internal nodes to the memory (write internal nodes)

<img src="CFSM.PNG" style="width: 500px;"/>

### Composition: Memory + Constructor 

We compose the Node Memory with the Huffman Constructor into one system so there is no input.

\**The diagram is taken from our CSE225 presentation*

## UCLID5 Basics
UCLID5 is a technique for system modelling and verification.

Main parts of a typical UCLID5 program:

<img src="./assets/basics.png" style="width: 400px;"/>

### Correlation Between Verilog and UCLID5

1. the init block describes registers after reset
2. the next block describes how registers update values

<img src="./assets/correlation.png" style="width: 500px;"/>

## Verification

### Deterministic Memory

- Fistly, we want to make sure that our model is correct
- We initialize the whole memory such that there is only one possible run
- We assert values at different state. If the assertion fails, UCLID5 prints out the counter examples
- And we correct our model based on the counter example

<img src="./assets/nond.png" style="width: 500px;"/>


### Non-Deterministic Memory

- Then we need to generalize the memory to allow for general verification
- Instead of initializing the whole memory, what really matters for Huffman is:
    - For all the alphabet characters, they don't have children
    - There are at least two characters with non-zero frequency
- Some initial values are irrelavent, because they will be overwritten before read
    - But we initialize them to zeros, so that UCLID5 has smaller space to explore

<img src="./assets/re_d_init.png" style="width: 700px;"/>

### What Properties Contribute to the Correctness of the system?

<img src="./assets/nond.png" style="width: 500px;"/>

### Diameter Analysis

- The algorithm always terminates and has an upper-bound for the run time.
- Given the initial memory size (number of characters), we can calculate the worst-case run time.
- Therefore, as long as the steps are large enough, bounded model checking accomplishes complete verification.

Assume the size of the alphabet is $n$
- Read through the alphabet and count the initial heap size: $O(n)$
- Heap construction: $O(n)$ *
- Huffman Construction: $O(n\log n)$

\**Reference: http://en.wikipedia.org/wiki/Binary_heap*


## Synthesis

UCLID5 uses cvc5 as the engine for function synthesis
- Provide the function signature and the specification to be met
- Does not seem to allow syntax information

### The Problem: Synthesizing the Truth table

Minheapify: a function that maintains the order between the parents and children. If the parent is larger than the children, we need to replace the parent with the smallest value.

```C++
smallest = parent;

if (left_child < smallest) {
    smallest = left_child;
}

if (right_child < smallest) {
    smallest = right_child;
}

return smallest;
```

For the hardware implementation, we want to get the smallest value within a single cycle. Then we need to analyze case by case, and write down the whole "truth table."

<img src="eq.PNG" style="width: 200px;"/>

```Verilog
				casez ({less1, less2, less3})
					{3'b000}:
						smallest_heap_index = right_child_heap_index;
					{3'b01?}:
						smallest_heap_index = heap_index;
					{3'b100}:
						smallest_heap_index = right_child_heap_index;
					{3'b101}:
						smallest_heap_index = left_child_heap_index;
					{3'b111}:
						smallest_heap_index = left_child_heap_index;
					default: 
						smallest_heap_index = 0;
				endcase
```

\**This diagram is taken from our CSE225 presentation*

Can we let the tool synthesize the case by case analysis for us?

<img src="./assets/sml_syn.png" style="width: 700px;"/>


Output in cvc5 grammar. It might be feasible if we directly use cvc5 and provide more syntax information.

```
(define-fun synth_get_smallest ((eq1 Bool) (eq2 Bool) (eq3 Bool) (val1 Int) (val2 Int) (val3 Int)) Int 
(let ((_let_1 (* (- 1) val3)))
(let ((_let_2 (+ val2 _let_1)))
(let ((_let_3 (>= _let_2 0)))
(let ((_let_4 (+ val1 _let_1)))
(let ((_let_5 (>= _let_4 0)))
(let ((_let_6 (+ val1 (* (- 1) val2))))
(let ((_let_7 (>= _let_6 1)))
(let ((_let_8 (= eq1 _let_7)))
(ite (and (= eq2 (not _let_5))
(= eq3 (not _let_3))
(or (and eq1 _let_7)
(and _let_8 (>= _let_4 1))))
(ite (and _let_8 (or (>= _let_2 1)
(not (>= _let_6 0))))
(ite (and (or _let_5 (not eq2))
(or _let_3 (not eq3)))
val3 (+ 1 val3))
val2) val1))))))))))
```

## What's Next

